In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scipy.io

from config import Config
import utils
import model as modellib
import visualize
from model import log
import prostate

%matplotlib inline

# GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


Using TensorFlow backend.


In [3]:
dataset_dir = "/scratch/wenyuan/Mask_RCNN_On_Pathology/Data_Pre_Processing/cedars-224"
held_out_set = 4
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
class InferenceConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    DETECTION_NMS_THRESHOLD = 1     
    MEAN_PIXEL = np.array(mean_pixel)

inference_config = InferenceConfig()
inference_config.display()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path, find a trained weights specified by epoch and held_out_set or find last trained weights

# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]
# model_path = model.find_specific(epoch = 9, held_out_set = 4)[1]
# model_path = model.find_specific(epoch = 160, held_out_set = held_out_set)[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)


Configurations:
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        1
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 193.97800579  120.89113632  183.79060979]
MINI_MASK_SHAPE                (56, 56)
NAME                           prostate
NUM_CLASSES                    4
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST

In [4]:
def instance_2_sementic(instance_mask, class_ids):
    try:
        h, w, d = instance_mask.shape
    except ValueError:
        mask = int(class_ids) * instance_mask 
        result_dict = {'ATmask': mask}
        return result_dict
    
    mask_map = {}
    for index, label in enumerate(class_ids):
        mask_map[str(label)] = np.logical_or(mask_map[str(label)], \
                                             instance_mask[:, :, index]) \
        if str(label) in mask_map.keys() else instance_mask[:, :, index]
    
    mask = np.zeros((h, w), dtype=np.int)

    for key in mask_map.keys():
        if (key != '0'):
            mask = mask + int(key) * mask_map[key] 
    result_dict = {'ATmask': mask}
    return result_dict

In [5]:
def prediction_2_sementic(mask, class_ids, scores):
    """ convert maskrcnn prediction to 1 single sementic mask
    """
    mask_map = {} # create a probability dict for each class
    for index, label in enumerate(class_ids):
        mask_map[str(label)] = \
        np.maximum(mask_map[str(label)], scores[index] * mask[:, :, index])\
        if str(label) in mask_map.keys() else scores[index] * mask[:, :, index]
    
    ## convert to h * w * num_classes probability map
    h, w, d = mask.shape
    for i in range(4):
        try:
            sementic_mask = \
            np.concatenate((sementic_mask, \
                            np.expand_dims(mask_map[str(i)], axis = -1)),axis = -1)\
            if (i != 0) else np.expand_dims(mask_map[str(i)], axis = -1)
        except KeyError:
            sementic_mask = np.concatenate((sementic_mask, np.zeros((h, w, 1))),axis = -1)\
            if (i != 0) else np.zeros((h, w, 1))
            
    sementic_mask_res = np.argmax(sementic_mask, axis = -1)
    return sementic_mask_res

In [6]:
def combine_2_whole_slide(patch_list, width_num, height_num):
    assert len(patch_list) == width_num * height_num, "Patch num doesn't match width_num * height_num!"
    row_level = []
    for i in range(height_num):
        cur_level = np.concatenate(patch_list[i * width_num : (i + 1) * width_num], axis = 1)
        row_level.append(cur_level)
    return np.concatenate(row_level, axis = 0)

In [7]:
def compute_mIOU(c_matrix, th):
    """ compute the mIOU based on the confusion matrix
    """
    num_class, _ = c_matrix.shape
    IOU = []
    sum_IOU = 0
    num_classes = 0
    below_th = False
    for i in range(num_class):
        p = c_matrix[i, i] / (sum(c_matrix[i, :]) + sum(c_matrix[:, i]) - c_matrix[i, i]) \
        if (sum(c_matrix[i, :]) + sum(c_matrix[:, i]) - c_matrix[i, i]) else 0
        IOU.append(p)
        if p != 0:
            sum_IOU += p
            num_classes += 1
    if sum_IOU / num_classes < th:
        below_th = True
    return np.mean(IOU), IOU, below_th

In [8]:
dataset_val = prostate.ProstateDataset()
_, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)
# val_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# exclude_list = [394, 185, 506, 289, 360, 7, 222, 406,\
#                152, 195, 219, 510, 196, 399, 237, 41,\
#                44, 274, 423, 432, 207, 371, 463, 482,\
#                176, 50, 494]
# val_list = [image for image in val_list if image not in exclude_list]
dataset_val.load_prostate(dataset_dir, val_list, mode = 16)
dataset_val.prepare()

In [9]:
height_crop = [128, 384]
width_crop = [128, 384]

for image_id in range(0, len(dataset_val.image_ids), 16):
    image_whole = []
    gt_mask_whole = []
    pred_mask_whole = []
    for i in range(16):
        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, inference_config,
                                   image_id + i, use_mini_mask=False)
        image_whole.append(image[height_crop[0] : height_crop[1], width_crop[0] : width_crop[1]])

        # Convert gt-instance mask to gt-sementic mask
        gt_sementic_mask = instance_2_sementic(gt_mask, gt_class_id)
        gt_mask_whole.append(gt_sementic_mask['ATmask'][height_crop[0] : height_crop[1], width_crop[0] : width_crop[1]])

        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        
        if np.argmax(r['tumor_probs']) == 1 and r['class_ids'].size != 0:
            result_sementic = prediction_2_sementic(r['masks'], r['class_ids'], r['scores'])
        else:
            result_sementic = np.zeros((image.shape[0], image.shape[1]))    
        
#         # Convert prediction to sementic mask
#         result_sementic = prediction_2_sementic(r['masks'], r['class_ids'], r['scores'])\
#         if r['class_ids'].size != 0 else np.zeros((image.shape[0], image.shape[1]))  
        
        pred_mask_whole.append(result_sementic[height_crop[0] : height_crop[1], width_crop[0] : width_crop[1]])
    img = combine_2_whole_slide(image_whole, 4, 4)
    ann = combine_2_whole_slide(gt_mask_whole, 4, 4)
    pred = combine_2_whole_slide(pred_mask_whole, 4, 4)
    c_matrix = confusion_matrix(np.ravel(ann), np.ravel(pred))
    if c_matrix.size != 16:
            """if the confusion matrix is not 4 by 4
            """
            unique_set = \
            np.union1d(np.unique(pred), \
                       np.unique(ann))
            xv, yv = np.meshgrid(unique_set, unique_set)
            temp = np.zeros((4, 4))
            temp[yv.astype(int), xv.astype(int)] = c_matrix
            c_matrix = temp 

    mIOU, IOU, below_th = compute_mIOU(c_matrix, th = 0.50)
    if below_th:
        print("Number of image is:", val_list[int(image_id / 16)])
        print("mIOU: ", mIOU, "IOU:", IOU)
    if image_id % (16 * 5):
        print("Done evaluating image : %d !" %image_id)

Done evaluating image : 16 !
Done evaluating image : 32 !
Number of image is: 154
mIOU:  0.311874938291 IOU: [0.59567049534779537, 0.0010048189221607288, 0.65082443889518071, 0]
Done evaluating image : 48 !
Done evaluating image : 64 !
Number of image is: 185
mIOU:  0.309682759514 IOU: [0.35626443337692332, 0, 0.88230558056617814, 0.00016102411336097581]
Done evaluating image : 96 !
Done evaluating image : 112 !
Done evaluating image : 128 !
Done evaluating image : 144 !
Done evaluating image : 176 !
Done evaluating image : 192 !
Done evaluating image : 208 !
Done evaluating image : 224 !
Done evaluating image : 256 !
Done evaluating image : 272 !
Done evaluating image : 288 !
Done evaluating image : 304 !
Done evaluating image : 336 !
Done evaluating image : 352 !
Done evaluating image : 368 !
Done evaluating image : 384 !
Number of image is: 7
mIOU:  0.207182060755 IOU: [0.12302101643555098, 0, 0.70570722658499863, 0]
Done evaluating image : 416 !
Done evaluating image : 432 !
Done e